In [1]:
import pandas as pd
import numpy as np
import os
import sys

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
cwd = os.getcwd()

education_folder_path = os.path.dirname(cwd)

sys.path.append(education_folder_path)

In [3]:
from global_functions import remove_spaces_in_columns, up_load_df

In [4]:
#בתי ספר וגנים מעיריית ירושלים
JLM=up_load_df(r'{}\background_files\jerusalem_muni'.format(education_folder_path),'מוסדות בירושלים 2020')
JLM = remove_spaces_in_columns(JLM)

In [ ]:
JLM.rename(columns={'סמל_חינוך': 'סמל_מוסד'}, inplace=True)

JLM = JLM.drop(columns=['שם_מוסד', 'coordinate_x', 'coordinate_y', 'אגף', 'פיקוח', 'סוג_חינוך', 'מעמד_משפטי', 'מספר_כיתות', "מס'_תלמידים_ז-ט", "מס'_תלמידים_י-יד", 'קוד_אזור_סטיסטי', 'תאור_אזור_סטטיסטי', 'Unnamed:_18', 'Unnamed:_19'])

JLM.head()

In [6]:
# פונקציה להמרת כיתות לאותיות למספרים
def get_grade_number(grade):
    grade_map = {
        'א': 1, 'ב': 2, 'ג': 3, 'ד': 4, 'ה': 5, 'ו': 6,
        'ז': 7, 'ח': 8, 'ט': 9, 'י': 10, 'יא': 11, 'יב': 12, 'יג': 13, 'יד': 14,
        '3': 3, '4': 4, '5': 5  # עבור גני ילדים
    }
    return grade_map.get(grade, np.nan)

In [7]:
# חישוב מספר הכיתות
JLM['מספר_כיתות'] = JLM.apply(lambda row: get_grade_number(row['עד_כיתה']) - get_grade_number(row['מכיתה']) + 1, axis=1)
JLM['מספר_כיתות']

# חלוקת התלמידים לפי כיתות
JLM['תלמידים_לכיתה'] = JLM['סה"כ_תלמידים'] / JLM['מספר_כיתות']

In [8]:
# יצירת עמודות עבור כל כיתה
grades = ['א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט', 'י', 'יא', 'יב', 'יג', 'יד']
for grade in grades:
    JLM[grade] = JLM.apply(lambda row: row['תלמידים_לכיתה'] if get_grade_number(row['מכיתה']) <= get_grade_number(grade) <= get_grade_number(row['עד_כיתה']) else 0, axis=1)

# JLM.head(600)

In [ ]:
# מיפוי הכיתות לפי שכבות גיל
ages = {
    'kgarden': [3, 4, 5],
    'ele': ['א', 'ב', 'ג', 'ד', 'ה', 'ו'],
    'mid': ['ז', 'ח', 'ט'],
    'high': ['י', 'יא', 'יב', 'יג', 'יד']
}

# פונקציה להמרת אותיות למספרים עבור השדה 'מכיתה' ו'עד כיתה'
def class_to_num(c):
    classes = {'א': 6, 'ב': 7, 'ג': 8, 'ד': 9, 'ה': 10, 'ו': 11, 'ז': 12, 'ח': 13, 'ט': 14, 'י': 15, 'יא': 16, 'יב': 17, 'יג': 18, 'יד': 19}
    return classes.get(c, c)

JLM['מכיתה_num'] = JLM['מכיתה'].apply(class_to_num)
JLM['עד_כיתה_num'] = JLM['עד_כיתה'].apply(class_to_num)

# פונקציה לחישוב מספר הכיתות לכל שכבת גיל
def calc_classes_for_age(row, age_group):
    # קבלת טווח הכיתות עבור כל שכבת גיל
    if age_group == 'kgarden':
        # גן ילדים
        return sum([1 for x in ages['kgarden'] if row['מכיתה_num'] <= x <= row['עד_כיתה_num']])
    elif age_group == 'ele':
        # יסודי
        return sum([1 for x in ages['ele'] if class_to_num(x) >= row['מכיתה_num'] and class_to_num(x) <= row['עד_כיתה_num']])
    elif age_group == 'mid':
        # חטיבת ביניים
        return sum([1 for x in ages['mid'] if class_to_num(x) >= row['מכיתה_num'] and class_to_num(x) <= row['עד_כיתה_num']])
    elif age_group == 'high':
        # על יסודי
        return sum([1 for x in ages['high'] if class_to_num(x) >= row['מכיתה_num'] and class_to_num(x) <= row['עד_כיתה_num']])
    return 0

# הוספת עמודות לכל שכבת גיל
JLM['kgarden_class'] = JLM.apply(calc_classes_for_age, axis=1, age_group='kgarden')
JLM['ele_class'] = JLM.apply(calc_classes_for_age, axis=1, age_group='ele')
JLM['mid_class'] = JLM.apply(calc_classes_for_age, axis=1, age_group='mid')
JLM['high_class'] = JLM.apply(calc_classes_for_age, axis=1, age_group='high')

JLM = JLM.drop(columns=['מכיתה_num', 'עד_כיתה_num'])

JLM.head(600)

In [11]:
# פונקציה לסיווג לפי שכבת גיל
def classify_students(row, grades_dict):
    # אם זה גני ילדים, נכניס את כל התלמידים לקטגוריה kgarden
    if row['שלב_חינוך'] == 'גני ילדים':
        return pd.Series({'kgarden_stu': row['סה"כ_תלמידים'], 'ele_stu': 0, 'mid_stu': 0, 'high_stu': 0})
    
    # חישוב מספר הכיתות בטווח
    from_grade = get_grade_number(row['מכיתה'])
    to_grade = get_grade_number(row['עד_כיתה'])
    
    # חלוקה לפי שכבות גיל
    ele_students = 0
    mid_students = 0
    high_students = 0

    # נבדוק באיזה טווח כיתות הערכים ונחלק את התלמידים לפי שכבת גיל
    if from_grade <= 6 and to_grade >= 1:  # יסודי
        ele_students = row['סה"כ_תלמידים'] * len([g for g in grades_dict['ele'] if from_grade <= get_grade_number(g) <= to_grade]) / (to_grade - from_grade + 1)
    
    if from_grade <= 9 and to_grade >= 7:  # חטיבת ביניים
        mid_students = row['סה"כ_תלמידים'] * len([g for g in grades_dict['mid'] if from_grade <= get_grade_number(g) <= to_grade]) / (to_grade - from_grade + 1)
    
    if from_grade >= 10:  # תיכון
        high_students = row['סה"כ_תלמידים'] * len([g for g in grades_dict['high'] if from_grade <= get_grade_number(g) <= to_grade]) / (to_grade - from_grade + 1)
        print(row['סמל_מוסד'])
        print(high_students)
    return pd.Series({'kgarden_stu': 0, 'ele_stu': ele_students, 'mid_stu': mid_students, 'high_stu': high_students})

In [ ]:
# המרת תלמידים לפי שכבות הגיל שהוגדרו
ages = {'kgarden':[3, 4, 5],'ele':['א', 'ב', 'ג', 'ד','ה','ו'],'mid':['ז', 'ח', 'ט'], 'high':['י', 'יא', 'יב', 'יג', 'יד']}

# הוספת עמודות של התלמידים לפי שכבת גיל
JLM[['kgarden_stu', 'ele_stu', 'mid_stu', 'high_stu']] = JLM.apply(lambda row: classify_students(row, ages), axis=1)

# תצוגת הדאטה פריים הסופי
# JLM.head()


In [21]:
# פונקציה לחישוב מספר הכיתות או הגנים
def calculate_classes(row, grades_dict):
    # גני ילדים
    if row['שלב_חינוך'] == 'גני ילדים':
        from_grade = get_grade_number(row['מכיתה'])
        to_grade = get_grade_number(row['עד_כיתה'])
        
        # אם יש ערך במספרי הכיתות ומספר התלמידים גדול מאפס
        if pd.notna(from_grade) and pd.notna(to_grade) and row['סה"כ_תלמידים'] > 0:
            num_grades = to_grade - from_grade + 1  # מספר הגנים
            kgarden_class = num_grades
        else:
            kgarden_class = 0
        return pd.Series({'kgarden_class': kgarden_class, 'ele_class': 0, 'mid_class': 0, 'high_class': 0})
    
    # חישוב מספר הכיתות בטווח
    from_grade = get_grade_number(row['מכיתה'])
    to_grade = get_grade_number(row['עד_כיתה'])
    
    # מספר הכיתות לכל שכבת גיל
    ele_class = 0
    mid_class = 0
    high_class = 0
    
    # חישוב מספר הכיתות בכל שכבת גיל
    if from_grade <= 6 and to_grade >= 1:  # יסודי (א-ו)
        ele_class = len([g for g in grades_dict['ele'] if from_grade <= get_grade_number(g) <= to_grade])
    
    if from_grade <= 9 and to_grade >= 7:  # חטיבת ביניים (ז-ט)
        mid_class = len([g for g in grades_dict['mid'] if from_grade <= get_grade_number(g) <= to_grade])
    
    if from_grade >= 10:  # על יסודי (י-יד)
        high_class = len([g for g in grades_dict['high'] if from_grade <= get_grade_number(g) <= to_grade])
        print(high_class)
    
    return pd.Series({'kgarden_class': 0, 'ele_class': ele_class, 'mid_class': mid_class, 'high_class': high_class})

In [ ]:
# הגדרת המילון עבור שכבות גיל
ages = {'kgarden':[3, 4, 5],'ele':['א', 'ב', 'ג', 'ד','ה','ו'],'mid':['ז', 'ח', 'ט'], 'high':['י', 'יא', 'יב', 'יג', 'יד']}

# חישוב הכיתות עבור כל מוסד
JLM[['kgarden_class', 'ele_class', 'mid_class', 'high_class']] = JLM.apply(lambda row: calculate_classes(row, ages), axis=1)

# הצגת התוצאה
JLM = JLM.drop(columns=['תלמידים_לכיתה'])

JLM.head(600)

JLM.to_excel('data.xlsx', index=False)